# Bright By Text: Outgoing Messages Analysis

## Setup and Data Import

In [1]:
# Widen notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import libraries necessary
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
from time import time
import datetime
from IPython.display import display
import re
%matplotlib inline
import psycopg2
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [3]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [17]:
# Query to get archived outbound messages for 2017-2018

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    smt.name as scheduled_message_tag
    
from 
(
    select *
    from archive.outbound_messages
    where extract(year from created_at) <= 2018
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join scheduled_messages sm 
on msm.scheduled_message_id = sm.id 
left join scheduled_message_to_tags smtt 
on sm.id = smtt.scheduled_message_id 
left join scheduled_message_tags smt 
on smtt.scheduled_message_tag_id = smt.id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2017_2018 = pd.read_sql_query(sql, conn)
conn = None

In [19]:
archived_messages_2017_2018

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,scheduled_message_tag
0,5ec45471-2146-456a-acb4-da0d41e6c5bf,2017-05-01 14:40:45.723794,Is this zipcode correct? If yes reply with 'Y...,00000000-0000-0000-0000-000000000002,signup,42cace4c-0d50-407f-8276-609079e560e9,2017-05-01 14:04:57.337456,2017-05-01 14:04:57.337456,NaT,Verizon Wireless,07860,Newton,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,2.000,None
1,00f97ed1-d95f-47a7-8738-93d2ec8be5f1,2017-05-01 18:22:11.956767,Is this zipcode correct? If yes reply with 'Y...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,47764722-ac32-4362-9a1a-ce4503f4cf84,2017-05-01 18:16:25.565976,2017-05-01 18:16:25.565976,NaT,Verizon Wireless,78245,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,1.000,None
2,ffb5de23-d828-4e3a-becf-8b10b80cefba,2017-05-01 18:34:33.615578,Unrecognized date of birth format. Please res...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,8f619c33-e411-460b-8bf8-ea3b4e668ea2,2017-05-01 18:31:47.699809,2017-05-01 18:31:47.699809,2018-04-30 14:46:49.920313,"T-Mobile USA, Inc.",None,None,nan,None,None,Text SignUp,deactivated,English,STOP,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,nan,None
3,00f97ed1-d95f-47a7-8738-93d2ec8be5f1,2017-05-02 21:13:27.420256,Is this zipcode correct? If yes reply with 'Y...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,1003849f-4d8e-430b-af34-e5e1c8d8b4f6,2017-05-02 21:10:55.406804,2017-05-02 21:10:55.406804,NaT,AT&T Wireless,78253,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,1.000,None
4,68c00142-67b6-4c14-93a3-948b425ccc25,2017-05-03 11:38:58.633927,Unrecognized date of birth format. Please res...,00000000-0000-0000-0000-000000000002,signup,d184f7d8-764c-4792-86ba-de17563b3a0b,2017-05-03 10:30:20.168807,2017-05-03 10:30:20.168807,2018-04-30 14:46:53.934041,Verizon Wireless,None,None,nan,None,None,Text SignUp,deactivated,English,STOP,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,nan,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622621,147f8eb2-fac8-4eeb-8d5e-75de98c8cf8c,2017-05-10 16:54:09.557283,"Our messages are research-based, created by to...",f92494a8-bcce-473f-a56e-0ae2515c827e,signup,f2a88950-aade-4ba5-9d3b-2009e7a0aff1,2017-05-10 16:52:45.961870,2017-05-10 16:52:45.961870,2018-04-30 14:47:09.681798,"Sprint Spectrum, L.P.",None,None,nan,None,None,Text SignUp,deactivated,English,Signup not Completed,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,nan,None
6622622,02ed28c1-126e-45d4-8a67-13cd84dee0ca,2017-05-10 16:56:20.921404,Bright by Text won't work unless we have a chi...,00000000-0000-0000-0000-000000000002,signup,723eca14-52bc-43d8-ac68-81c86185bed2,2017-05-10 16:52:17.587055,2017-05-10 16:52:17.587055,NaT,"T-Mobile USA, Inc.",11421,Woodhaven,-5.000,America/New_York,New York,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,None
6622623,ab1fdb70-fcc5-46a6-8954-df793fca59ee,2017-05-10 16:58:14.322014,Thanks! We'll be sending you resources from Vr...,00000000-0000-0000-0000-000000000002,signup,723eca14-52bc-43d8-ac68-81c86185bed2,2017-05-10 16:52:17.587055,2017-05-10 16:52:17.587055,NaT,"T-Mobile USA, Inc.",11421,Woodhaven,-5.000,America/New_York,New York,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,

In [20]:
# Download the archived messages dataset

archived_messages_2017_2018.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2017_2018.csv', index=False)  

In [25]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [8]:
# Query to get archived outbound messages for 2019

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    smt.name as scheduled_message_tag
    
from 
(
    select *
    from archive.outbound_messages
    where extract(year from created_at) = 2019
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join scheduled_messages sm 
on msm.scheduled_message_id = sm.id 
left join scheduled_message_to_tags smtt 
on sm.id = smtt.scheduled_message_id 
left join scheduled_message_tags smt 
on smtt.scheduled_message_tag_id = smt.id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2019 = pd.read_sql_query(sql, conn)
conn = None

In [9]:
archived_messages_2019

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,scheduled_message_tag
0,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:10:18.585484,Make a summer reading challenge with your chil...,d4d696af-37ff-427e-8035-2cb1bd630495,national,93794ab3-dd7d-4fe5-8612-046e703f5e2f,2017-06-19 11:39:06.879671,2015-11-23 07:09:00.000000,NaT,Verizon Wireless,80535,Laporte,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,2.000,None
1,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:10:20.526938,Make a summer reading challenge with your chil...,d4d696af-37ff-427e-8035-2cb1bd630495,national,9434b99f-7bc7-4184-93de-c8ddbd08ef77,2017-06-19 13:07:04.551823,2016-09-13 13:02:00.000000,NaT,"Sprint Spectrum, L.P.",80234,Denver,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,None
2,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:10:20.999294,Make a summer reading challenge with your chil...,24bf3729-46b3-47fa-ba19-6786f208758b,national,9459fdc1-3ff6-4c26-8458-dd7956ec9c96,2019-01-09 03:42:51.483925,2019-01-09 03:42:51.483925,NaT,"Metro PCS, Inc.",76106,Fort Worth,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,1.000,None
3,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:10:23.326747,Make a summer reading challenge with your chil...,87d65e72-a157-4cd9-b7a4-f1e6ff052f42,national,94fa8a3e-51e8-47cc-bcf5-bc0e6efe0ee0,2017-09-05 19:51:43.069602,2017-09-05 19:51:43.069602,NaT,AT&T Wireless,47802,Terre Haute,-5.000,America/Indiana/Indianapolis,Indiana,Text SignUp,activated,English,None,"WFYI - Indianapolis, IN",2018-08-28 21:00:49.021838,True,Indiana,1.000,None
4,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:10:23.748315,Make a summer reading challenge with your chil...,62c3f855-e888-41a5-ab0b-1dd69a01dd3d,national,95122527-cd47-4eaa-a702-bdd8a4dc01a9,2018-03-29 21:11:21.127436,2018-03-29 21:11:21.127436,NaT,"T-Mobile USA, Inc.",78572,Mission,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,United Way of Southern Cameron County - Browns...,2018-08-28 20:59:07.931146,True,Texas,2.000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10143599,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:08:44.129622,Make a summer reading challenge with your chil...,00000000-0000-0000-0000-000000000001,national,78b03c74-65c7-41f8-9a9e-37e31a6e93e1,2018-03-25 00:50:33.411973,2018-03-25 00:50:33.411973,NaT,Verizon Wireless,85308,Glendale,-7.000,America/Phoenix,Arizona,Web SignUp,activated,English,None,ArizonaPBS,2018-03-25 21:32:18.027322,True,Arizona,1.000,None
10143600,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:08:44.336855,Make a summer reading challenge with your chil...,d4d696af-37ff-427e-8035-2cb1bd630495,national,78b130d5-6b89-45f7-8a1a-3c362a9aefc0,2018-11-14 19:08:07.012958,2018-11-14 19:08:07.012958,NaT,Verizon Wireless,08016,Burlington,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,2.000,None
10143601,31bd8232-0648-47f1-a2c9-50010da3e573,2019-06-18 18:08:45.255129,Make a summer reading challenge with your chil...,fde09cdf-a4db-462c-9f06-0eaf341e418c,national,79103e9c-71ab-43af-906d-30b2acce507e,2018-06-05 17:07:10.505999,2018-06-05 17:07:10.505999,NaT,"Metro PCS, Inc.",79907,El Paso,-7.000,America/Denver,Texas,CSV Upload,activated,English,None,KCOS - El Paso,2018-08-28 21:01:59.594441,True,Texas,1.000,No

In [13]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [14]:
# Query to get archived outbound messages for 2020

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    smt.name as scheduled_message_tag
    
from 
(
    select *
    from archive.outbound_messages
    where extract(year from created_at) = 2020
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join scheduled_messages sm 
on msm.scheduled_message_id = sm.id 
left join scheduled_message_to_tags smtt 
on sm.id = smtt.scheduled_message_id 
left join scheduled_message_tags smt 
on smtt.scheduled_message_tag_id = smt.id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2020 = pd.read_sql_query(sql, conn)
conn = None

In [15]:
# Download the archived messages dataset

archived_messages_2020.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2020.csv', index=False)  

In [4]:
# Load the messages datasets
messages_2017_2018 = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2017_2018.csv', low_memory = False)
messages_2019 = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2019.csv', low_memory = False)
messages_2020 = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2020.csv', low_memory = False)

In [5]:
messages = pd.concat([messages_2017_2018, messages_2019, messages_2020])

In [7]:
# Download the full messages dataset

messages.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_2017_2020.csv', index=False)  

Note that this table only includes outbound messages, and does not include inbound messages.